# Combining several cost functions

<!-- {{ add_binder_block(page) }} -->

## Introduction

In `ruptures`, change point detection procedures make use of only one cost function.
The choice of the cost function is critical as it is related to the type of change to find. 
For instance, [CostL2](../user-guide/costs/costl2.md) can detect shifts in the mean, [CostNormal](../user-guide/costs/costnormal.md) can detect shifts in the mean and the covariance structure, [CostAR](../user-guide/costs/costautoregressive.md) can detect shifts in the auto-regressive structure, etc.

However, in many settings, several types of changes co-exist in the same signal and a single cost function is not able to spot all changes simultaneously.
To cope with this issue, a procedure to merge several cost functions has been introduced [[Katser2021]](#Katser2021).
In a nutshell, a number of costs can be combined to yield an aggregated cost function which is sensitive to several types of changes.
The aggregated cost can then be used with any search method (such as the [window search method](../user-guide/detection/window.md)) to create change point detection algorithm.

This example illustrates the aggregation procedure, also referred to as an ensemble model.
Here, only [dynamic programming](../user-guide/detection/dynp.md) is considered, but all other search methods (e.g. [window sliding search method](../user-guide/detection/window.md), [PELT](../user-guide/detection/pelt.md), [binary segmentation](../user-guide/detection/binseg.md)) could be used.
In addition, the number of changes is assumed to be known by the user.
More details can be found in the original paper introducing the cost aggregation procedure [[Katser2021]](#Katser2021).

## Setup

First, we make the necessary imports and generate a multivariate toy signal which contains mean shifts and linear changes (i.e. changes in the linear relationship between the dimensions of the signal). 

In [ ]:
import matplotlib.pyplot as plt  # for display purposes
import numpy as np

import ruptures as rpt  # our package
from ruptures.metrics import hamming

# generate a signal
n_samples, n_dims, sigma = 500, 3, 4
n_bkps = 10  # number of breakpoints

# to make it more complex, we concatenate two different signals together
signal_constant, bkps_constant = rpt.pw_constant(
    n_samples=n_samples // 2, n_features=n_dims, n_bkps=n_bkps // 2, noise_std=sigma
)

signal_linear, bkps_linear = rpt.pw_linear(
    n_samples=n_samples // 2,
    n_features=n_dims - 1,
    n_bkps=n_bkps - n_bkps // 2 - 1,
    noise_std=0,
)

signal = np.r_[signal_constant, signal_linear]
bkps_true = sorted(bkps_constant + list(np.array(bkps_linear) + n_samples // 2))

# z-normalization
signal = (signal - signal.mean(axis=0)) / signal.std(axis=0)

The following plot shows the signal and the true change points (changes occur when the background colour shifts from blue to pink and vice versa).

In [ ]:
# show the signal
fig, ax_array = rpt.display(signal, bkps_true)

## Using one cost function at a time

Recall that two types of changes are present in the previous signal: mean shifts and linear changes.
The most adapted costs for these types are:

- [CostL2](../user-guide/costs/costl2.md) (for mean shifts),
- [CostLinear](../user-guide/costs/costlinear.md) (for linear changes).

The following cell shows that using a single cost function is not enough to detect all changes.

In [ ]:
list_of_cost_functions = ["l2", "linear"]

for cost_str in list_of_cost_functions:
    # Compute the changes
    algo = rpt.Dynp(model=cost_str).fit(signal)
    predicted_bkps = algo.predict(n_bkps=n_bkps)
    # Display the prediction
    fig, (ax,) = rpt.display(signal[:, 0], bkps_true, predicted_bkps)
    ax.margins(x=0)
    ax.set_title(
        f"Cost {cost_str} (Hamming error: {hamming(bkps_true, predicted_bkps):.2f})"
    )

In the previous plots, only the first dimension of the signal is shown. The hamming error is also added. It goes from 0 to 1, the lower the better.

The first plot shows that the [CostL2](../user-guide/costs/costl2.md) allows the [dynamic programming detector](../user-guide/detection/dynp.md) to find the true breakpoints only on the first part of the signal. Whereas the second plot tells us that the [CostLinear](../user-guide/costs/costlinear.md) is less helpful on the entire signal.
The overall performance of the two detectors is not satisfying.

## Using several cost functions: the ensemble method

Roughly, the ensemble method scales each individual score and aggregates them to get a new score which will allow us to get a better prediction of the changepoints.

In [[Katser2021]](#Katser2021), the authors perform multiple tests on several scaling and aggregation functions on two datasets. It turned out that the *MinMax* scaling function and the *Min* aggregation function worked best.

The *MinMax* function is defined as follows:
For $s$ a timeseries, 

$$
\textit{MinMax}(s)_i = \frac{s_i - \max_{j}{s_j}}{\max_{j}{s_j} - \min_{j}{s_j}}
$$

The *Min* function is defined as follows:
Let $s^n, n \in \{1, ..., N\}$ be $N$ timeseries, we have
$$ 
\textit{Min}((s^n)_{n \in \{1, ..., N\}})_i = \min_{p \in \{1, ..., N\}}{s^p_i} 
$$

In [ ]:
def min_max_scaling(array):
    return (array - np.max(array, axis=0)) / (
        np.max(array, axis=0) - np.min(array, axis=0) + 1e-8
    )


def min_aggregation(array):
    return array.min(axis=1)

A new detector needs to be build in order to combine the [CostL2](../user-guide/costs/costl2.md) and the [CostLinear](../user-guide/costs/costlinear.md). Here is how it can be build:

In [ ]:
r"""Dynamic programming"""
from functools import lru_cache

from ruptures.utils import sanity_check
from ruptures.costs import cost_factory
from ruptures.exceptions import BadSegmentationParameters


class DynpEnsemble(rpt.Dynp):

    """Find optimal change points using dynamic programming with more than one cost.

    Given a segment model, it computes the best partition for which the
    sum of errors is minimum.
    """

    def __init__(self, models, min_size=2, jump=5, params=None):
        """Creates a Dynp instance.

        Args:
            models (list[str], optional): segment model, ex: ["l1"], ["l2", "rbf"], ...
            min_size (int, optional): minimum segment length.
            jump (int, optional): subsample (one every *jump* points).
            params (dict, optional): a dictionary of dictionnaries of parameters for the cost instances. One dictionnary of parameters per cost instance.
        """
        self.model_names = models
        self.costs = []
        self.min_size = min_size
        for model in models:
            if params is None:
                self.costs.append(cost_factory(model=model))
            else:
                self.costs.append(cost_factory(model=model, **params[model]))
            self.min_size = max(self.min_size, self.costs[-1].min_size)
        self.jump = jump
        self.n_samples = None

    @lru_cache(maxsize=None)
    def seg(self, start, end, n_bkps):
        """Recurrence to find the optimal partition of signal[start:end].

        This method is to be memoized and then used.

        Args:
            start (int): start of the segment (inclusive)
            end (int): end of the segment (exclusive)
            n_bkps (int): number of breakpoints

        Returns:
            dict: {(start, end): cost value, ...}
        """
        jump, min_size = self.jump, self.min_size

        if n_bkps == 0:
            cost = [
                self.costs[idx_cost].error(start, end)
                for idx_cost in range(len(self.costs))
            ]
            return {(start, end): np.array(cost)}
        elif n_bkps > 0:
            # Let's fill the list of admissible last breakpoints
            multiple_of_jump = (k for k in range(start, end) if k % jump == 0)
            admissible_bkps = list()
            for bkp in multiple_of_jump:
                n_samples = bkp - start
                # first check if left subproblem is possible
                if sanity_check(
                    n_samples=n_samples,
                    n_bkps=n_bkps - 1,
                    jump=jump,
                    min_size=min_size,
                ):
                    # second check if the right subproblem has enough points
                    if end - bkp >= min_size:
                        admissible_bkps.append(bkp)

            assert (
                len(admissible_bkps) > 0
            ), "No admissible last breakpoints found.\
             start, end: ({},{}), n_bkps: {}.".format(
                start, end, n_bkps
            )

            # Compute the subproblems
            sub_problems = list()
            for bkp in admissible_bkps:
                left_partition = self.seg(start, bkp, n_bkps - 1)
                right_partition = self.seg(bkp, end, 0)
                tmp_partition = dict(left_partition)
                tmp_partition[(bkp, end)] = right_partition[(bkp, end)]
                sub_problems.append(tmp_partition)

            # compute the matrix of cost, row = sub_problem, column = model
            cost_matrix = [
                np.sum(list(sub_problem.values()), axis=0)
                for sub_problem in sub_problems
            ]

            aggregated_costs = min_aggregation(min_max_scaling(np.array(cost_matrix)))

            # Find the optimal partition
            return sub_problems[np.argmin(aggregated_costs)]

    def fit(self, signal) -> "DynpEnsemble":
        """Create the cache associated with the signal.

        Dynamic programming is a recurrence; intermediate results are cached to speed up
        computations. This method sets up the cache.

        Args:
            signal (array): signal. Shape (n_samples, n_features) or (n_samples,).

        Returns:
            self
        """
        # clear cache
        self.seg.cache_clear()
        # update some params
        [self.costs[idx_cost].fit(signal) for idx_cost in range(len(self.costs))]
        self.n_samples = signal.shape[0]
        return self

    def predict(self, n_bkps):
        """Return the optimal breakpoints.

        Must be called after the fit method. The breakpoints are associated with the signal passed
        to [`fit()`][ruptures.detection.dynp.Dynp.fit].

        Args:
            n_bkps (int): number of breakpoints.

        Raises:
            BadSegmentationParameters: in case of impossible segmentation
                configuration

        Returns:
            list: sorted list of breakpoints
        """
        # raise an exception in case of impossible segmentation configuration
        if not sanity_check(
            n_samples=self.costs[0].signal.shape[0],
            n_bkps=n_bkps,
            jump=self.jump,
            min_size=self.min_size,
        ):
            raise BadSegmentationParameters
        partition = self.seg(0, self.n_samples, n_bkps)
        bkps = sorted(e for s, e in partition.keys())
        return bkps

The newly made `DynpEnsemble` class is ready to be used.

In [ ]:
# create the ensemble change point detector
algo = DynpEnsemble(["l2", "linear"])
algo.fit(signal)

ensemble_predicted_bkps = algo.predict(n_bkps=n_bkps)

In [ ]:
# Display the prediction
fig, (ax,) = rpt.display(signal[:, 0], bkps_true, ensemble_predicted_bkps)
ax.margins(x=0)
_ = ax.set_title(
    f"Cost ensemble (Hamming error: {hamming(bkps_true, ensemble_predicted_bkps):.2f})"
)

## Conclusion

Through this example, we have seen how to build an ensemble model to detect changepoints in a few lines of code.

This example is using a dynamic programming search method, the algorithm for ensemble models using other search methods like binary segmentation or window search are given in the paper [Katser2021](#Katser2021).

## Authors

This example notebook has been authored by [Théo VINCENT](https://github.com/theovincent) and edited by [Olivier Boulant](https://github.com/oboulant) and [Charles Truong](https://github.com/deepcharles).


## References

<a id="Katser2021">[Katser2021]</a>
Katser, I., Kozitsin, V., Lobachev, V., & Maksimov, I. (2021). Unsupervised Offline Changepoint Detection Ensembles. Applied Sciences, 11(9), 4280.